# Implementation of DEBIAS-M

We save the results of DEBIAS-M as csv table, then load it in R for following analysis.

In [ ]:
from debiasm import DebiasMClassifier
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder

In [ ]:
# code for binary covariate

count = pd.read_csv("count",header = 0,index_col=0)

# sample by taxa 
count = count.to_numpy().T

meta = pd.read_csv("meta",header = 0,index_col=0)

# convert batch variable and sample covariate to numerical values
# Y is the target covariate of interest
le = LabelEncoder()
batch = le.fit_transform(meta["batch"])
Y = le.fit_transform(meta["Y"])

X_with_batch = np.hstack((batch[:, np.newaxis], count))

# set referece batch
val_inds = batch==0
X_train, X_val = X_with_batch[~val_inds], X_with_batch[val_inds]
Y_train, Y_val = Y[~val_inds], Y[val_inds]

### Run DEBIAS-M, using standard sklearn object methods
dmc = DebiasMClassifier(x_val=X_val) ## give it the held-out inputs to account for
                                    ## those domains shifts while training
dmc.fit(X_train, Y_train)

X_debiassed = dmc.transform(X_with_batch)
df1 = pd.DataFrame(X_debiassed)
df1.to_csv("debiasm_res.csv")

In [ ]:
# code for continuous covariate
from debiasm import DebiasMRegressor

count = pd.read_csv("count",header = 0,index_col=0)

# sample by taxa 
count = count.to_numpy().T

meta = pd.read_csv("meta",header = 0,index_col=0)

le = LabelEncoder()
batch = le.fit_transform(meta["batch"])

X_with_batch = np.hstack((batch[:, np.newaxis], count))
# target variable of interest
Y = np.array(meta["Y"])

val_inds = batch==0
X_train, X_val = X_with_batch[~val_inds], X_with_batch[val_inds]
Y_train, Y_val = Y[~val_inds], Y[val_inds]

### Run DEBIAS-M, using standard sklearn object methods
dmc = DebiasMRegressor(x_val=X_val) ## give it the held-out inputs to account for
                                    ## those domains shifts while training
dmc.fit(X_train, Y_train)

X_debiassed = dmc.transform(X_with_batch)
df2 = pd.DataFrame(X_debiassed)
df2.to_csv("debiasm_res_2.csv")